In [ ]:
from secretnote.formal.locations import (
    SPUFieldType,
    SPUProtocolKind,
    SymbolicPYU,
    SymbolicSPU,
    SymbolicWorld,
)

sym_world = SymbolicWorld(world=frozenset(("alice", "bob")))
sym_alice = SymbolicPYU("alice")
sym_bob = SymbolicPYU("bob")
sym_spu = SymbolicSPU(
    world=frozenset(("alice", "bob")),
    protocol=SPUProtocolKind.SEMI2K,
    field=SPUFieldType.FM128,
    fxp_fraction_bits=0,
)

In [ ]:
import ray
from secretnote.formal.locations import SFConfigSimulation, PortBinding

ray.shutdown()

sym_world.reify(SFConfigSimulation())

alice = sym_alice.reify()
bob = sym_bob.reify()
spu = sym_spu.reify(
    alice=PortBinding(announced_as="127.0.0.1:32767"),
    bob=PortBinding(announced_as="127.0.0.1:32768"),
)

In [ ]:
from secretnote.instrumentation.sdk import create_profiler, setup_tracing

setup_tracing()

profiler = create_profiler()
profiler.start()

In [ ]:
"""Yao's Millionaires' Problem in SecretFlow"""

import jax
import secretflow

key = jax.random.PRNGKey(42)


def make_money(seed: jax.random.KeyArray, generation: int) -> jax.Array:
    for _ in range(generation):
        seed, subkey = jax.random.split(seed)
    return jax.random.randint(seed, shape=(), minval=10**6, maxval=10**9)


def compare(a: jax.Array, b: jax.Array) -> jax.Array:
    return a > b


balance_alice = alice(make_money)(key, 3)
balance_bob = bob(make_money)(key, 2)

balance_alice = balance_alice.to(spu)
balance_bob = balance_bob.to(spu)

alice_is_richer = spu(compare)(balance_alice, balance_bob)

alice_is_richer = secretflow.reveal(alice_is_richer)
print(f"{alice_is_richer=}")

account_alice, account_bob = secretflow.reveal((balance_alice, balance_bob))

print(f"{account_alice=}")
print(f"{account_bob=}")

In [ ]:
from secretnote.display import visualize_run

profiler.stop()
visualize_run(profiler)